# Stream Ciphers

treats the plaintext as a stream and encrypts the bits individually

smaller and faster 

assumed to be more efficient

encryption and decryption are the same procedure -XOR

they can reach very high level of secrecy

One time pod: key as long as the plaintext, uniformly distributed in the key space, key must be used only once -> information theory-wise secure

Security in stream ciphers relies on key stream

OTP is unpractical because stream ciphers rely on random number generators

-maybe talk a bit about RNGs- 

RNG must be reproducible and unpredictable



In [7]:
from bits import Bits
from lfsr import LFSR, berlekamp_massey
from bitgenerator import AlternatingStep

## LFSR

one of the main building blocks of PRNGs

In [8]:
lfsr = LFSR({1, 2, 5}, 0b000)
bits = lfsr.cycle()
print(bits)

0


In [5]:
lfsr = LFSR({1, 2, 5}, 0b00101)
output = lfsr.run_steps(10)

In [6]:
output

Bits([True, False, True, False, False, True, True, True, False, True])

In [ ]:
bits = lfsr.cycle(state=0b00101)

## Berlekamp-Massey Algorithm

In [ ]:
with open('binary_sequence.bin', 'rb') as f:
        binary_sequence = f.read()

In [26]:
binary_sequence[:50]

b'\xbb`\xef\x067\xae\xd0K"Vd]#Q\xeb\x02~<\xe6C\xbe\xed5\xd0\xec\xada\xe8\x89h\xf3\xbdFc\x96\xb5\x8e\xb0\x03\xabVFY#\xd1\xeb">\xb5\xe4'

In [28]:
bits = Bits(binary_sequence)

In [34]:
poly = berlekamp_massey(bits)
linear_complexity = max(poly) if poly else 0

print("Shortest feedback polynomial degrees:", poly)
print("Linear complexity:", linear_complexity)

Shortest feedback polynomial degrees: {0, 7, 18}
Linear complexity: 18


## Alternating Step Generator

In [2]:
alt_step = AlternatingStep()

In [4]:
bits = alt_step.run(25)
bits

[True,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True]